# **Asistente conversacional con gpt-4o-mini**

## **1. Configuración del entorno del cuaderno**
---
En esta celda se importan las librerías necesarias y se configuran las claves API para diferentes servicios. También se instalan las dependencias de LangChain y sus integraciones con OpenAI, Groq, Google GenAI y HuggingFace.(Este código se explico con detalle en el primer cuaderno)

In [ ]:
# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
!pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
!pip install langchain-openai -qU
!pip install langchain-groq -qU
!pip install langchain-google-genai -qU
!pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

## **2. Configuración del modelo**
---
Aquí se configura el modelo de OpenAI (`gpt-4o-mini`) con la clave API y un parámetro de temperatura para controlar la creatividad de las respuestas.

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

## 3. **Preparamos las plantillas**
---
Se definen dos plantillas: una para el sistema (instrucciones del tutor) y otra para el usuario (mensaje del estudiante). Estas plantillas se utilizarán para generar prompts estructurados.

In [ ]:
# Definir las plantillas
template_sistema = """\
Eres un tutor amigable de {idioma}. Tu trabajo es:
1. Mantener una conversación simple con el estudiante
2. Corregir errores básicos de gramática y ortografía si los hay
3. Ser paciente y motivador
"""

template_usuario = """\
El estudiante dice: {mensaje_usuario}

Por favor, responde de manera natural y si hay algún error gramatical,
corrígelo sutilmente en tu respuesta.
"""

## **4. Creamos el ChatPromptTemplate**
---
Se crea un `ChatPromptTemplate` utilizando las plantillas definidas. Se muestra una alternativa para crear el mismo template, ambas formas son válidas.  

Primera forma:  
Se utilizan clases específicas (SystemMessagePromptTemplate y HumanMessagePromptTemplate) para crear los mensajes del sistema y del usuario, respectivamente.  

Segunda forma:  
Se utiliza una tupla (role, template) para definir los mensajes, donde role es el tipo de mensaje (por ejemplo, "system" o "human") y template es la plantilla de texto.

Es más conciso y directo, ya que no requiere instanciar clases específicas.

In [ ]:
# Crear el ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(template_sistema),
    HumanMessagePromptTemplate.from_template(template_usuario),
])

# Podria ser asi ???? explicar

# Crear el ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template_sistema),
    ("human", template_usuario),
])

## **5. Función para chatear en un idioma específico**
---
Esta función toma un mensaje del usuario y un idioma (en principio esta fijado en español), formatea el prompt, invoca el modelo de lenguaje y devuelve la respuesta. Es el núcleo del asistente conversacional.

In [ ]:
def chatear_en_idioma(mensaje_usuario, idioma="español"):
    """
    Función simple para procesar mensajes del usuario y obtener respuestas
    1. Recibe el mensaje del usuario
    2. Lo inyecta en el chatprompttemplate
    3. Invoca el modelo con el
    4. Devuelve la respuesta
    """
    # Formatear el prompt con el mensaje del usuario y el idioma
    prompt = chat_prompt.format(
        mensaje_usuario=mensaje_usuario,
        idioma=idioma
    )

    # Obtener la respuesta del modelo
    respuesta = llm.invoke(prompt)

    return respuesta.content

##**6.  Interfaz de chat interactivo.**  
---  
Se implementa un bucle interactivo que permite al usuario chatear con el asistente. El usuario puede escribir "salir" para terminar la conversación. El asistente responde corrigiendo errores y manteniendo una conversación natural.

In [ ]:
    print("¡Bienvenido al Chatbot de Aprendizaje de Idiomas!")
    print("Escribe 'salir' para terminar la conversación")

    while True:
        # Obtener entrada del usuario
        mensaje = input("\nTú: ")

        if mensaje.lower() == 'salir':
            print("¡Hasta luego! Gracias por practicar.")
            break

        # Obtener y mostrar la respuesta
        respuesta = chatear_en_idioma(mensaje)
        print("\nTutor:", respuesta)